In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


books=pd.read_csv('./books.csv')
books.head()

,Unnamed: 0,Hunger,Potter,Twilight,Mockingbird,Gatsby
0,0,False,True,False,True,True
1,1,False,True,True,False,True
2,2,False,False,False,True,False
3,3,False,True,False,False,True
4,4,False,False,False,False,True


In [2]:
# Compute support for Hunger and Potter
supportHP = np.logical_and(books['Hunger'], books['Potter']).mean()

# Compute support for Hunger and Twilight
supportHT = np.logical_and(books['Hunger'], books['Twilight']).mean()

# Compute support for Potter and Twilight
supportPT = np.logical_and(books['Potter'], books['Twilight']).mean()

# Print support values
print("Hunger Games and Harry Potter: %.2f" % supportHP)
print("Hunger Games and Twilight: %.2f" % supportHT)
print("Harry Potter and Twilight: %.2f" % supportPT)

Hunger Games and Harry Potter: 0.12
Hunger Games and Twilight: 0.09
Harry Potter and Twilight: 0.14


In [3]:
# Compute support for Potter and Twilight
supportPT = np.logical_and(books['Potter'], books['Twilight']).mean()

# Compute support for Potter
supportP = books['Potter'].mean()

# Compute support for Twilight
supportT = books['Twilight'].mean()

# Compute confidence for both rules
confidencePT = supportPT / supportP
confidenceTP = supportPT / supportT

# Print results
print('{0:.2f}, {1:.2f}'.format(confidencePT, confidenceTP))

0.29, 0.55


In [4]:
# Compute support for Potter and Twilight
supportPT = np.logical_and(books['Potter'], books['Twilight']).mean()

# Compute support for Potter
supportP = books['Potter'].mean()

# Compute support for Twilight
supportT = books['Twilight'].mean()

# Compute lift
lift = supportPT / (supportP * supportT)

# Print lift
print("Lift: %.2f" % lift)

Lift: 1.15


In [5]:
# Compute support for Potter AND Hunger
supportPH = np.logical_and(books['Potter'], books['Hunger']).mean()

# Compute support for Potter
supportP = books['Potter'].mean()

# Compute support for NOT Hunger
supportnH = 1.0 - books['Hunger'].mean()

# Compute support for Potter and NOT Hunger
supportPnH = supportP - supportPH

# Compute and print conviction for Potter -> Hunger
conviction = supportP * supportnH / supportPnH
print("Conviction: %.2f" % conviction)

Conviction: 0.92


In [6]:
def conviction(antecedent, consequent):
	# Compute support for antecedent AND consequent
	supportAC = np.logical_and(antecedent, consequent).mean()

	# Compute support for antecedent
	supportA = antecedent.mean()

	# Compute support for NOT consequent
	supportnC = 1.0 - consequent.mean()

	# Compute support for antecedent and NOT consequent
	supportAnC = supportA - supportAC

    # Return conviction
	return supportA * supportnC / supportAnC

In [7]:
twilight = books['Twilight']
hunger = books['Hunger']
potter = books['Potter']
# Compute conviction for twilight -> potter and potter -> twilight
convictionTP = conviction(twilight, potter)
convictionPT = conviction(potter, twilight)

# Compute conviction for twilight -> hunger and hunger -> twilight
convictionTH = conviction(twilight, hunger)
convictionHT = conviction(hunger, twilight)

# Compute conviction for potter -> hunger and hunger -> potter
convictionPH = conviction(potter, hunger)
convictionHP = conviction(hunger, potter)

# Print results
print('Harry Potter -> Twilight: ', convictionPT)
print('Twilight -> Potter: ', convictionTP)

Harry Potter -> Twilight:  1.0534570072738598
Twilight -> Potter:  1.1550539077290998


Great job! Notice that the conviction metric for if Potter then Twilight and if Twilight then Potter are both above 1, indicating that they are both viable rules.

In [8]:
# Compute the support of Twilight and Harry Potter
supportT = books['Twilight'].mean()
supportP = books['Potter'].mean()

# Compute the support of both books
supportTP = np.logical_and(books['Twilight'], books['Potter']).mean()

# Complete the expressions for the numerator and denominator
numerator = supportTP - supportT*supportP
denominator = max(supportTP*(1-supportT), supportT*(supportP-supportTP))

# Compute and print Zhang's metric
zhang = numerator / denominator
print(zhang)

0.17231567178855997


In [9]:
# Define a function to compute Zhang's metric
def zhang(antecedent, consequent):
	# Compute the support of each book
	supportA = antecedent.mean()
	supportC = consequent.mean()

	# Compute the support of both books
	supportAC = np.logical_and(antecedent, consequent).mean()

	# Complete the expressions for the numerator and denominator
	numerator = supportAC - supportA*supportC
	denominator = max(supportAC*(1-supportA), supportA*(supportC-supportAC))

	# Return Zhang's metric
	return numerator / denominator

In [10]:
rules = pd.DataFrame({'antecedents': {0: 'Potter',
  1: 'Twilight',
  2: 'Mockingbird',
  3: 'Gatsby',
  4: 'Potter',
  5: 'Potter',
  6: 'Potter',
  7: 'Mockingbird',
  8: 'Gatsby',
  9: 'Mockingbird'},
 'consequents': {0: 'Hunger',
  1: 'Hunger',
  2: 'Hunger',
  3: 'Hunger',
  4: 'Twilight',
  5: 'Mockingbird',
  6: 'Gatsby',
  7: 'Twilight',
  8: 'Twilight',
  9: 'Gatsby'}})
itemsets = [['Potter', 'Hunger'],
 ['Twilight', 'Hunger'],
 ['Mockingbird', 'Hunger'],
 ['Gatsby', 'Hunger'],
 ['Potter', 'Twilight'],
 ['Potter', 'Mockingbird'],
 ['Potter', 'Gatsby'],
 ['Mockingbird', 'Twilight'],
 ['Gatsby', 'Twilight'],
 ['Mockingbird', 'Gatsby']]
rules

,antecedents,consequents
0,Potter,Hunger
1,Twilight,Hunger
2,Mockingbird,Hunger
3,Gatsby,Hunger
4,Potter,Twilight
5,Potter,Mockingbird
6,Potter,Gatsby
7,Mockingbird,Twilight
8,Gatsby,Twilight
9,Mockingbird,Gatsby


In [11]:
# Define an empty list for Zhang's metric
zhangs_metric = []
# Loop over lists in itemsets
for itemset in itemsets:
    # Extract the antecedent and consequent columns
	antecedent = books[itemset[0]]
	consequent = books[itemset[1]]
    
    # Complete Zhang's metric and append it to the list
	zhangs_metric.append(zhang(antecedent, consequent))
    
# Print results
rules['zhang'] = zhangs_metric
print(rules)

   antecedents  consequents     zhang
0       Potter       Hunger -0.306049
1     Twilight       Hunger  0.109357
2  Mockingbird       Hunger -0.525436
3       Gatsby       Hunger -0.550446
4       Potter     Twilight  0.245118
5       Potter  Mockingbird -0.065537
6       Potter       Gatsby -0.165572
7  Mockingbird     Twilight -0.319008
8       Gatsby     Twilight -0.370875
9  Mockingbird       Gatsby  0.466460


In [22]:
rules = pd.read_csv('./rules.csv', index_col=0)
rules.shape

(150, 9)

In [24]:
# Preview the rules DataFrame using the .head() method
rules = pd.read_csv('./rules.csv', index_col=0)
(rules.head())
rules['zhang'] = [-0.25285758994840396,
 -0.30604947931146775,
 0.11937338356510399,
 0.10935743194389685,
 -0.4598255098198574,
 -0.5254355556640548,
 -0.558994108961381,
 -0.550446260321376,
 0.24511760920374562,
 0.17231567178855997,
 -0.06542106983331514,
 -0.06553734857107217,
 -0.16557153311492417,
 -0.12822711011631466,
 -0.31900842047456696,
 -0.24808679356499785,
 -0.3585893145188873,
 -0.37087511216484337,
 0.46646026374035804,
 0.3464334775117851,
 0.4252122584465518,
 0.17530192655349097,
 0.13331661083051322,
 0.1682693511631984,
 0.30559051961250494,
 0.501257238646754,
 0.21047170024137699,
 -0.043231796603186254,
 -0.2305989114403802,
 -0.25571271085966285,
 -0.07030954311976204,
 0.36404698508226757,
 -0.16812997789228457,
 0.2344587203977496,
 -0.2636115459669152,
 -0.3144232048920239,
 0.4260504041844983,
 -0.2007874370658842,
 0.32011536573392246,
 -0.19592919965784225,
 0.10918514946962384,
 0.14460408684546605,
 -0.29773689297736894,
 0.38924273536926046,
 -0.32166450793396784,
 0.30953159754023296,
 0.07463648843751217,
 0.10375030202616405,
 0.39541200066856097,
 -0.3799456553842519,
 0.1585389397274025,
 0.28818712968856963,
 -0.49269631446040424,
 -0.5372473587632838,
 0.5226896611793398,
 0.20327296202270942,
 0.13466129604916827,
 0.28278736074893646,
 -0.06926017957620512,
 -0.10886423357664213,
 0.48805360239575113,
 0.14141349207369616,
 -0.20009065176770957,
 0.046023665881256406,
 0.265835888747032,
 0.48155103099468305,
 0.05401607585127569,
 -0.233707161070055,
 0.3570102555601858,
 0.013053989195068144,
 0.3696625464406866,
 0.6159872807561703,
 0.020335172897310145,
 0.39532876906673375,
 0.2222956103618181,
 -0.280473871264973,
 0.3134215650491651,
 0.5666727347198834,
 -0.29916776382560667,
 0.2843926393402252,
 0.42220466218885816,
 0.3256641256641256,
 0.0031284601292452664,
 0.18170489195280382,
 0.48811349017534583,
 0.5679510810093541,
 0.21055856309538792,
 0.24571309638952485,
 0.5518334446010434,
 0.5133023476475341,
 0.2500638569604085,
 0.005673396371070795,
 0.6016191765121863,
 0.5358181514290489,
 -0.15124047859354928,
 0.38884739855613637,
 0.31973962595189115,
 0.1531857771672961,
 0.5498493376776942,
 0.12055943092918922,
 0.4680035923473348,
 0.5170499358645099,
 0.12588176410891702,
 0.5090021497100377,
 0.21729965653634867,
 0.6007091118097584,
 0.5069379422928069,
 -0.19354473698736008,
 0.2744488489253405,
 0.30974242255239315,
 0.35650887573964496,
 -0.22730720401714918,
 0.50481477285601,
 -0.021650056552636562,
 0.5362451888030849,
 0.6523192707340408,
 -0.023267014506048886,
 0.5230030578934025,
 -0.28225558683838087,
 0.6598882844730021,
 0.5731634034059142,
 0.367270022892159,
 0.1966083730789612,
 0.3391139058732835,
 0.3143557600210421,
 0.08966329506796272,
 0.6563910787682771,
 -0.25387298369396033,
 0.6402924116133435,
 0.674163783160323,
 -0.27576162155601397,
 0.6267548363353742,
 0.12691257856910323,
 0.5894729985017263,
 0.44113714347514044,
 0.2692363515727051,
 0.25240863944676367,
 0.05804572409010005,
 0.29946178007165747,
 0.34313275149288003,
 0.6628048800293634,
 0.537441205990593,
 -0.08411214953271025,
 -0.08001823777499142,
 0.5555804207185581,
 0.5465813179189513,
 0.6330937690690893,
 0.5523596028996478,
 0.07108491064819707,
 0.3355513513124671]

In [25]:


# Select the subset of rules with antecedent support greater than 0.05
rules = rules[rules['antecedent support'] > 0.05]

# Select the subset of rules with a consequent support greater than 0.02
rules = rules[rules['consequent support'] > 0.02]

# Select the subset of rules with a conviction greater than 1.01
rules = rules[rules['conviction'] > 1.01]

# Print remaining rules
(rules)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhang
2,frozenset({'Hunger'}),frozenset({'Twilight'}),0.319130,0.256770,0.089193,0.279486,1.088468,0.007249,1.031527,0.119373
3,frozenset({'Twilight'}),frozenset({'Hunger'}),0.256770,0.319130,0.089193,0.347363,1.088468,0.007249,1.043260,0.109357
8,frozenset({'Potter'}),frozenset({'Twilight'}),0.477516,0.256770,0.140621,0.294485,1.146881,0.018009,1.053457,0.245118
9,frozenset({'Twilight'}),frozenset({'Potter'}),0.256770,0.477516,0.140621,0.547654,1.146881,0.018009,1.155054,0.172316
18,frozenset({'Mockingbird'}),frozenset({'Gatsby'}),0.476522,0.295155,0.186087,0.390511,1.323070,0.045439,1.156452,0.466460
...,...,...,...,...,...,...,...,...,...,...
143,"frozenset({'Potter', 'Gatsby'})","frozenset({'Mockingbird', 'Twilight'})",0.127702,0.098261,0.024348,0.190661,1.940360,0.011800,1.114168,-0.080018
145,"frozenset({'Gatsby', 'Twilight'})","frozenset({'Mockingbird', 'Potter'})",0.053540,0.219503,0.024348,0.454756,2.071754,0.012596,1.431465,0.546581
146,frozenset({'Mockingbird'}),"frozenset({'Potter', 'Gatsby', 'Twilight'})",0.476522,0.034161,0.024348,0.051095,1.495687,0.008069,1.017845,0.633094
147,frozenset({'Potter'}),"frozenset({'Mockingbird', 'Gatsby', 'Twilight'})",0.477516,0.036273,0.024348,0.050989,1.405678,0.007027,1.015506,0.552360


In [27]:
# Set the lift threshold to 1.5
rules = rules[rules['lift'] > 1.5]

# Set the conviction threshold to 1.0
rules = rules[rules['conviction'] > 1.0]

# Set the threshold for Zhang's rule to 0.65
rules = rules[rules['zhang'] > 0.65]

# Print rule
(rules[['antecedents','consequents']])

,antecedents,consequents
126,"frozenset({'Mockingbird', 'Hunger'})","frozenset({'Gatsby', 'Twilight'})"
129,"frozenset({'Hunger', 'Gatsby'})","frozenset({'Mockingbird', 'Twilight'})"
140,"frozenset({'Mockingbird', 'Potter'})","frozenset({'Gatsby', 'Twilight'})"
